# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import axes3d
import numpy as np
import pandas as pd
import seaborn as sns
from sqlalchemy import func
from collections import OrderedDict

sns.set_style("whitegrid")

PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix() + '/src'
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [12]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import compute_affine_transformation, get_affine_transformation, list_coms

from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, read_image, write_image


In [5]:
def absolute_sum(l):
    la = np.array(l)
    #nabs = np.abs(np.array(la))
    return np.sum(la, axis=0)


## Data Collection

In [7]:
atlas_structures = list_coms('Atlas')
allen_structures = list_coms('Allen')

common_keys = atlas_structures.keys() & allen_structures.keys()
atlas_src = np.array([atlas_structures[s] for s in common_keys])
allen_src = np.array([allen_structures[s] for s in common_keys])
print(atlas_src.shape)

(37, 3)


In [28]:
errors = []
df_list = []
matrix = get_affine_transformation('Atlas')
for structure in common_keys:
    atlas0 = np.array(atlas_structures[structure])
    allen0 = np.array(allen_structures[structure]) 
    transformed = apply_affine_transform(atlas0, matrix)
    transformed = [round(x,2) for x in transformed]
    difference = [round(a - b, 2) for a, b in zip(transformed, allen0)]
    errors.append(difference)
    row = [structure, atlas0, allen0, transformed, difference]
    df_list.append(row)
result = absolute_sum(errors)
print(result)

[-1352.61  1771.02   609.96]


In [40]:
print('Affine transformation matrix')
print(np.array2string(matrix, separator=', '))

Affine transformation matrix
[[ 9.42279538e-01, -1.30121150e-03, -1.50092611e-02,  1.67617121e+02],
 [ 5.07770162e-02,  1.19394497e+00,  8.60586632e-03, -8.14323008e+01],
 [ 4.02430133e-03,  7.16889453e-03,  1.12982209e+00, -6.62364827e+01],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]


In [42]:
columns = ['structure', 'atlas0', 'allen0', 'transformed', 'difference']
df = pd.DataFrame(df_list, columns=columns)
columns = ['atlas0', 'allen0', 'transformed', 'difference']
df.index.name = 'Index'
df.sort_values(by=['structure'], inplace=True)
from IPython.display import HTML
HTML(df.to_html(index=False))

structure,atlas0,allen0,transformed,difference
3N_L,"[775.63, 391.54, 557.44]","[910.25, 379.44, 552.1]","[889.6, 430.23, 569.5]","[-20.65, 50.79, 17.4]"
3N_R,"[775.63, 391.54, 582.56]","[910.34, 379.4, 586.44]","[889.22, 430.44, 597.88]","[-21.12, 51.04, 11.44]"
4N_L,"[809.24, 396.73, 545.23]","[958.86, 377.78, 543.25]","[921.45, 438.02, 555.88]","[-37.41, 60.24, 12.63]"
4N_R,"[809.24, 396.73, 594.77]","[958.68, 377.48, 595.29]","[920.7, 438.45, 611.85]","[-37.98, 60.97, 16.56]"
5N_L,"[866.79, 502.13, 432.89]","[1019.3, 528.65, 409.16]","[977.22, 565.82, 429.94]","[-42.08, 37.17, 20.78]"
5N_R,"[866.79, 502.13, 707.11]","[1019.34, 528.55, 729.28]","[973.11, 568.18, 739.76]","[-46.23, 39.63, 10.48]"
6N_L,"[926.23, 512.4, 533.79]","[1077.12, 521.48, 529.09]","[1031.71, 581.97, 544.25]","[-45.41, 60.49, 15.16]"
6N_R,"[926.23, 512.4, 606.21]","[1077.14, 521.46, 609.41]","[1030.62, 582.59, 626.07]","[-46.52, 61.13, 16.66]"
7N_L,"[939.45, 615.73, 455.77]","[1085.27, 677.54, 434.0]","[1045.2, 705.34, 456.9]","[-40.07, 27.8, 22.9]"
7N_R,"[939.45, 615.73, 684.23]","[1085.26, 677.53, 704.52]","[1041.77, 707.31, 715.02]","[-43.49, 29.78, 10.5]"


In [ ]:
def prepare_table(brains, person_id, input_type_id, save_path):
    df_save = prepare_table_for_save(
        brains,
        person_id=person_id,
        input_type_id=input_type_id
    )
    df_save.to_csv(save_path, index=False)
    
    df = prepare_table_for_plot(
        brains,
        person_id=person_id,
        input_type_id=input_type_id
    )

    return df_save, df

def get_brain_coms(brains, person_id, input_type_id):
    brain_coms = {}
    for brain in brains:
        brain_coms[brain] = query_brain_coms(
            brain,
            person_id=person_id,
            input_type_id=input_type_id
        )
        # A temporary hack: for ('DK55', corrected), use ('DK55', aligned)
        if (brain, input_type_id) == ('DK55', 2):
            brain_coms[brain] = query_brain_coms(
                brain,
                person_id=person_id,
                input_type_id=4
            )
    return brain_coms

def prepare_table_for_save(brains, person_id, input_type_id):
    brain_coms = get_brain_coms(brains, person_id, input_type_id)

    data = {}
    data['name'] = []
    for s in common_structures:
        for c in ['dx', 'dy', 'dz', 'dist']:
            data['name'] += [f'{s}_{c}']
    for brain in brain_coms.keys():
        data[brain] = []
        offset = [brain_coms[brain][s] - atlas_coms[s]
                  if s in brain_coms[brain] else [np.nan, np.nan, np.nan]
                  for s in common_structures]
        offset = np.array(offset)
        scale = np.array([10, 10, 20])
        dx, dy, dz = (offset * scale).T
        dist = np.sqrt(dx * dx + dy * dy + dz * dz)
        for dx_i, dy_i, dz_i, dist_i in zip(dx, dy, dz, dist):
            data[brain] += [dx_i, dy_i, dz_i, dist_i]
    df = pd.DataFrame(data)

    return df

def prepare_table_for_plot(brains, person_id, input_type_id):
    brain_coms = get_brain_coms(brains, person_id, input_type_id)

    df = pd.DataFrame()
    for brain in brain_coms.keys():
        offset = [brain_coms[brain][s] - atlas_coms[s]
                  if s in brain_coms[brain] else [np.nan, np.nan, np.nan]
                  for s in common_structures]
        offset = np.array(offset)
        scale = np.array([10, 10, 20])
        dx, dy, dz = (offset * scale).T
        dist = np.sqrt(dx * dx + dy * dy + dz * dz)

        df_brain = pd.DataFrame()

        data = {}
        data['structure'] = common_structures
        data['value'] = dx
        data['type'] = 'dx'
        df_brain = df_brain.append(pd.DataFrame(data), ignore_index=True)

        data = {}
        data['structure'] = common_structures
        data['value'] = dy
        data['type'] = 'dy'
        df_brain = df_brain.append(pd.DataFrame(data), ignore_index=True)

        data = {}
        data['structure'] = common_structures
        data['value'] = dz
        data['type'] = 'dz'
        df_brain = df_brain.append(pd.DataFrame(data), ignore_index=True)

        data = {}
        data['structure'] = common_structures
        data['value'] = dist
        data['type'] = 'dist'
        df_brain = df_brain.append(pd.DataFrame(data), ignore_index=True)

        df_brain['brain'] = brain
        df = df.append(df_brain, ignore_index=True)
    return df

In [ ]:
def plot(df, ymin, ymax, ystep, title):
    fig, ax = plt.subplots(2, 1, figsize=(16, 12), dpi=200)
    
    sns.boxplot(ax=ax[0], x="structure", y="value", hue="type", data=df)
    ax[0].xaxis.grid(True)
    ax[0].set_xlabel('Structure')
    ax[0].set_ylabel('um')
    ax[0].set_title('full dynamic range')
    
    sns.boxplot(ax=ax[1], x="structure", y="value", hue="type", data=df)
    ax[1].xaxis.grid(True)
    ax[1].set_ylim(ymin, ymax)
    ax[1].yaxis.set_ticks(np.arange(ymin, ymax + 1, ystep))
    ax[0].set_xlabel('Structure')
    ax[1].set_ylabel('um')
    ax[1].set_title('zoom in')
    
    fig.suptitle(title, y=0.92)
    plt.show()
    return fig

figs = []

## Rigid Alignment Error

Rigid alignment error is computed as follows:
1. Anotomist manually annotate the COMs for each brain.
2. Computer finds the best transformation between atlas COMs and **anotomist's manual COMs**. The transformation is restricted to rigid + uniform scaling.
3. Using the transformation, the **anotomist's manual COMs** are brought to the atlas space.
4. The errors between the 2 sets of COMs are calculated, and displayed in the following plots.

The errors for a single structure are quantified by 4 numbers: dx, dy, dz, dist. (dx, dy, dz) are the offset. dist is the corresponding distance of the offset.

In [ ]:
brain_coms = get_brain_coms(brains_to_examine,person_id=28,
    input_type_id=4)

In [ ]:
# person is bili, input_type is aligned
df_save, df = prepare_table(
    brains_to_examine,
    person_id=28,
    input_type_id=4,
    save_path='../data/rigid-alignment-error.csv'
)
df_save.head()

In [ ]:
fig = plot(df, -1000, 1000, 100, 'Rigid Alignment Error')
figs.append(fig)

## Rigid Alignment Error After Correction

After reviewing the rigid alignment error plots, Beth manually re-annotatted the significant outliers as a correction. With this updated data, we compute the alignment error again using the same method.

In [ ]:
# person is bili, input_type is corrected
df_save, df = prepare_table(
    brains_to_examine,
    person_id=28,
    input_type_id=2,
    save_path='../data/rigid-alignment-error-after-correction.csv'
)
df_save.head()

In [ ]:
fig = plot(df, -1000, 1000, 100, 'Rigid Alignment Error After Correction')
figs.append(fig)

## Rough Alignment Error

**Rough alignment** is an **automatic method** to find the best 3D affine transformation between 2 brains, solely based on the thumbnail-resolution gray value images. Rough alignment is planned to be the first step of an automatic pipeline, which defines the starting points for Kui's automatic detection method.

We start with an anotomist manually annotate the COMs extensively for one brain (DK52).

Rough alignment error is computed as follows:
1. Computer finds the best 3D affine transformation between DK52 and the brain, as determined by aligning the gray value images.
2. Using the transformation, the DK52 COMs are brought to that brain's space. And we call it the **rough COMs** of the brain.
3. Computer finds the best transformation between atlas COMs and **rough COMs**. The transformation is restricted to rigid + uniform scaling.
4. Using the transformation, the **rough COMs** are brought to the atlas space.
5. The errors between the 2 sets of COMs are calculated, and displayed in the following plots.

The errors for a single structure are quantified by 4 numbers: dx, dy, dz, dist. (dx, dy, dz) are the offset. dist is the corresponding distance of the offset.

In [ ]:
# person is ed, input_type is aligned
df_save, df = prepare_table(
    brains_to_examine,
    person_id=1,
    input_type_id=4,
    save_path='../data/rough-alignment-error.csv'
)
df_save.head()

In [ ]:
fig = plot(df, -1000, 1000, 100, 'Rough Alignment Error')
figs.append(fig)

## Generate Report

In [ ]:
with PdfPages('../data/alignment-error.pdf') as pdf:
    for fig in figs:
        pdf.savefig(fig)